In [3]:
import pandas as pd
import numpy as np
from natsort import index_natsorted
import json

Chapter0: Data Pre-processing
1. Get information on the json file

In [4]:
with open("mlda_data.json") as f:
    data = json.load(f)

clip_counts = 0

for i in data.keys(): # video id
    for j in data[i]["clip"].keys(): # clips
        for k in data[i]["clip"][j]["scene_split"]:
            clip_counts += 1

print(len(data))
print(clip_counts)

11239
933677


2. Convert json into csv

In [21]:
df = pd.DataFrame()

video_id = []
video_ext = []
span_start = []
span_end = []
fps = []
scene_cut_start = []
scene_cut_end = []
clip_id = []
caption = []
clip_frames = []

for i in data.keys(): # for each video id
    temp_df = pd.DataFrame()
    for j in data[i]["clip"]: # for each video clips
        temp_end = []
        for k in range(len(data[i]["clip"][j]["scene_split"])):  # for each scenes
            video_id.append(i)
            video_ext.append(j)
            span_start.append("'" + data[i]["clip"][j]["span"][0])
            span_end.append("'" + data[i]["clip"][j]["span"][1])
            fps.append(data[i]["clip"][j]["fps"])
            scene_cut_start.append(data[i]["clip"][j]["scene_split"][k]["scene_cut"][0])
            scene_cut_end.append(data[i]["clip"][j]["scene_split"][k]["scene_cut"][1])
            temp_end.append(data[i]["clip"][j]["scene_split"][k]["scene_cut"][1])
            clip_id.append(data[i]["clip"][j]["scene_split"][k]["clip_id"])
            caption.append(data[i]["clip"][j]["scene_split"][k]["caption"])
        
        frames = max(temp_end)

        for k in range(len(data[i]["clip"][j]["scene_split"])):
            clip_frames.append(frames + 1)
           
df["video_id"] = video_id
df["video_ext"] = video_ext 
df["span_start"] = span_start
df["span_end"] = span_end
df["fps"] = fps
df["scene_cut_start"] = scene_cut_start
df["scene_cut_end"] = scene_cut_end
df["clip_id"] = clip_id
df["caption"] = caption
df["clip_frames"] = clip_frames
df.head(20)

,video_id,video_ext,span_start,span_end,fps,scene_cut_start,scene_cut_end,clip_id,caption,clip_frames
0,gpAkjSy_8iY,gpAkjSy_8iY.0.mp4,'00:00:00.030,'00:00:07.740,30.0,0,-1,gpAkjSy_8iY.0_0,a man in a suit and tie standing next to a new...,-1
1,gpAkjSy_8iY,gpAkjSy_8iY.8.mp4,'00:01:46.479,'00:01:57.009,30.0,9,37,gpAkjSy_8iY.8_1,a street that has power lines down on it,337
2,gpAkjSy_8iY,gpAkjSy_8iY.8.mp4,'00:01:46.479,'00:01:57.009,30.0,37,101,gpAkjSy_8iY.8_2,a woman standing in the middle of a flooded st...,337
3,gpAkjSy_8iY,gpAkjSy_8iY.8.mp4,'00:01:46.479,'00:01:57.009,30.0,101,153,gpAkjSy_8iY.8_3,a group of people standing around a car in a f...,337
4,gpAkjSy_8iY,gpAkjSy_8iY.8.mp4,'00:01:46.479,'00:01:57.009,30.0,153,337,gpAkjSy_8iY.8_4,bill patterson is interviewed on a newscast,337
5,gpAkjSy_8iY,gpAkjSy_8iY.8.mp4,'00:01:46.479,'00:01:57.009,30.0,0,9,gpAkjSy_8iY.8_0,a picture of a city street with power lines in...,337
6,gpAkjSy_8iY,gpAkjSy_8iY.10.mp4,'00:02:14.739,'00:02:22.380,30.0,204,206,gpAkjSy_8iY.10_1,an airplane flying through the dark sky,206
7,gpAkjSy_8iY,gpAkjSy_8iY.10.mp4,'00:02:14.739,'00:02:22.380,30.0,0,204,gpAkjSy_8iY.10_0,a picture of a man in a suit and tie,206
8,gpAkjSy_8iY,gpAkjSy_8iY.9.mp4,'00:01:57.009,'00:02:14.739,30.0,328,558,gpAkjSy_8iY.9_1,a picture of a man in a suit and tie,558
9,gpAkjSy_8iY,gpAkjSy_8iY.9.mp4,'00:01:57.009,'00:02:14.739,30.0,0,328,gpAkjSy_8iY.9_0,a fire on the left and a photo of the same fir...,558


In [22]:
# using natural sorting to sort the ids (to sort numbers following string)
sorted_indices = index_natsorted(df[['video_ext', 'clip_id']].to_numpy())

# Reorder the DataFrame using the sorted indices
new_df = df.iloc[sorted_indices]

In [23]:
new_df.to_csv("info.csv", index=False) # intermediate save
df = new_df

3. Additional processing to get more information

In [24]:
# Function to convert timestamp to seconds with milliseconds
def timestamp_to_seconds(timestamp):
    hours, minutes, seconds_ms = map(str, timestamp.split(':'))

    if '.' in seconds_ms:
        seconds, milliseconds = map(int, seconds_ms.split('.'))
    else:
        seconds = int(seconds_ms)
        milliseconds = 0

    hours = hours[1:]

    return int(hours) * 3600 + int(minutes) * 60 + seconds + milliseconds / 1000

In [25]:
def find_duration(span_start, span_end):
    start_time = timestamp_to_seconds(span_start)
    end_time = timestamp_to_seconds(span_end)
    return end_time - start_time

In [26]:
# filtering rows where fps is larger than 23 only
df = df[df['fps'] >= 23]

In [27]:
# find duration of each scenes
df['duration'] = df.apply(lambda row: find_duration(row['span_start'], row['span_end']), axis=1)    
df

C:\Users\ACER\AppData\Local\Temp\ipykernel_16264\2253615893.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['duration'] = df.apply(lambda row: find_duration(row['span_start'], row['span_end']), axis=1)


,video_id,video_ext,span_start,span_end,fps,scene_cut_start,scene_cut_end,clip_id,caption,clip_frames,duration
0,gpAkjSy_8iY,gpAkjSy_8iY.0.mp4,'00:00:00.030,'00:00:07.740,30.00000,0,-1,gpAkjSy_8iY.0_0,a man in a suit and tie standing next to a new...,-1,7.710
34,gpAkjSy_8iY,gpAkjSy_8iY.1.mp4,'00:00:07.740,'00:00:23.789,30.00000,0,-1,gpAkjSy_8iY.1_0,a news anchor standing in front of a fire warn...,-1,16.049
10,gpAkjSy_8iY,gpAkjSy_8iY.2.mp4,'00:00:23.789,'00:00:35.969,30.00000,0,106,gpAkjSy_8iY.2_0,a man in a suit and tie standing next to a new...,419,12.180
11,gpAkjSy_8iY,gpAkjSy_8iY.2.mp4,'00:00:23.789,'00:00:35.969,30.00000,106,133,gpAkjSy_8iY.2_1,a truck in front of a large fire,419,12.180
12,gpAkjSy_8iY,gpAkjSy_8iY.2.mp4,'00:00:23.789,'00:00:35.969,30.00000,133,201,gpAkjSy_8iY.2_2,a firefighter using a hose to put out a fire,419,12.180
...,...,...,...,...,...,...,...,...,...,...,...
933675,msKdwfqKgro,msKdwfqKgro.0.mp4,'00:00:00.000,'00:00:05.520,30.00000,0,-1,msKdwfqKgro.0_0,a man standing in front of two cars in a garage,-1,5.520
933673,msKdwfqKgro,msKdwfqKgro.1.mp4,'00:00:05.520,'00:00:16.500,30.00000,0,-1,msKdwfqKgro.1_0,a man standing in front of two cars in a garage,-1,10.980
933674,msKdwfqKgro,msKdwfqKgro.2.mp4,'00:00:16.500,'00:00:21.710,30.00000,0,-1,msKdwfqKgro.2_0,a man standing in front of a green and a red m...,-1,5.210
933670,mxv7EotS-OU,mxv7EotS-OU.0.mp4,'00:00:02.683,'00:00:14.160,29.97003,0,-1,mxv7EotS-OU.0_0,an image of a man being rescued from the water,-1,11.477


In [28]:
# calculate number of frames of each scenes
df['total_frames'] = (df['duration'] * df['fps']).astype(int)

C:\Users\ACER\AppData\Local\Temp\ipykernel_16264\2693850716.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['total_frames'] = (df['duration'] * df['fps']).astype(int)


In [29]:
# Remove rows where 'scene_cut_end' column has a value of -1
# long scenes will have many transitions, hence low quality
df = df[df['scene_cut_end'] != -1]

df

,video_id,video_ext,span_start,span_end,fps,scene_cut_start,scene_cut_end,clip_id,caption,clip_frames,duration,total_frames
10,gpAkjSy_8iY,gpAkjSy_8iY.2.mp4,'00:00:23.789,'00:00:35.969,30.0,0,106,gpAkjSy_8iY.2_0,a man in a suit and tie standing next to a new...,419,12.18,365
11,gpAkjSy_8iY,gpAkjSy_8iY.2.mp4,'00:00:23.789,'00:00:35.969,30.0,106,133,gpAkjSy_8iY.2_1,a truck in front of a large fire,419,12.18,365
12,gpAkjSy_8iY,gpAkjSy_8iY.2.mp4,'00:00:23.789,'00:00:35.969,30.0,133,201,gpAkjSy_8iY.2_2,a firefighter using a hose to put out a fire,419,12.18,365
13,gpAkjSy_8iY,gpAkjSy_8iY.2.mp4,'00:00:23.789,'00:00:35.969,30.0,201,237,gpAkjSy_8iY.2_3,a fire truck and two men standing next to it,419,12.18,365
14,gpAkjSy_8iY,gpAkjSy_8iY.2.mp4,'00:00:23.789,'00:00:35.969,30.0,237,331,gpAkjSy_8iY.2_4,a group of firefighters standing next to a fence,419,12.18,365
...,...,...,...,...,...,...,...,...,...,...,...,...
933629,hnzm4IBXnt8,hnzm4IBXnt8.22.mp4,'00:07:07.430,'00:07:37.240,25.0,705,748,hnzm4IBXnt8.22_14,a man hugging a woman dressed as a princess,849,29.81,745
933631,hnzm4IBXnt8,hnzm4IBXnt8.22.mp4,'00:07:07.430,'00:07:37.240,25.0,748,794,hnzm4IBXnt8.22_15,a group of people dressed in costumes on a stage,849,29.81,745
933634,hnzm4IBXnt8,hnzm4IBXnt8.22.mp4,'00:07:07.430,'00:07:37.240,25.0,794,849,hnzm4IBXnt8.22_16,a sign that says caba boi in russian,849,29.81,745
933667,mqgYOA3KN4c,mqgYOA3KN4c.0.mp4,'00:00:00.000,'00:00:39.650,25.0,0,686,mqgYOA3KN4c.0_0,a person using a pen to draw a picture,992,39.65,991


In [40]:
df['bias'] = df['clip_frames'] - df['total_frames']
df = df[df['bias'] >= 0]
# df.head(20)

C:\Users\ACER\AppData\Local\Temp\ipykernel_16264\1886759273.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bias'] = df['clip_frames'] - df['total_frames']


In [46]:
df['actual_scene_cut_start'] = df['scene_cut_start'] - df['bias']
df['actual_scene_cut_end'] = df['scene_cut_end'] - df['bias']

# make sure that start is at least 0
df['actual_scene_cut_start'] = df['actual_scene_cut_start'].apply(lambda x: max(0, x))
df = df[df['actual_scene_cut_end'] > 0]

# number of frames in scene should be at least its fps, and lower than 500
df['scene_frames'] = df['actual_scene_cut_end'] - df['actual_scene_cut_start']
df = df[df['scene_frames'] >= df['fps']]
df = df[df['scene_frames'] <= 500]

In [48]:
df.to_csv("output.csv", index=False) # final of pre-processing